In [1]:
# 다음 사이트의 연령대별 랭킹 뉴스 정보 수집하기
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math 
import pandas as pd  
import xlwt 
import random
import os

#Step 2. 사용자에게 검색 기간과 파일 저장 폴더 정보를 입력 받습니다.
print("=" *100)
print("이 프로그램은 교육용 목적으로 서진수가 개발했으며 절대 상업용으로 사용해선 안됩니다")
print("또한 이 프로그램으로 특정 회사에 피해를 줄 경우")
print("모든 책임은 프로그램 사용자에게 있음을 미리 공지합니다")
print("이 프로그램과 관련한 문의는 seojinsu@gmail.com 으로 보내주세요^^")
print("=" *100)
print("다음 사이트 랭킹 뉴스중 연령별 랭킹 기사 정보 수집용 웹크롤러 by 서진수 ")
print("=" *100)
print("\n")

st_date = input('1.조회 시작일을 입력하세요(예:20160101) :  ')
en_date = input('2.조회 종료일을 입력하세요(예:20161231) :  ')

f_dir=input('3.파일이 저장될 경로만 쓰세요;기본값:c:\\temp\\): ')
if f_dir=='' :
    f_dir="c:\\temp\\"
    
print("\n")
print("요청하신 정보를 수집중이니 잠시만 기다려 주세요~~^^")
print("\n")

#Step 3. 결과를 저장할 파일과 폴더명을 설정합니다
s_time = time.time( )

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

s_year=st_date[0:4]
e_year= en_date[0:4]

os.makedirs(f_dir+'다음뉴스_20대이하여성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_20대이하남성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_30대여성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_30대남성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_40대여성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_40대남성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_50대이상여성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_50대이상남성_'+s_year+'-'+s)
os.makedirs(f_dir+'다음뉴스_전체_'+s_year+'-'+s)

ff_20_w_name=f_dir+'다음뉴스_20대이하여성_'+s_year+'-'+s+'\\'+'다음뉴스_20대이하여성_'+s_year+'-'+s+'.txt'
ff_20_m_name=f_dir+'다음뉴스_20대이하남성_'+s_year+'-'+s+'\\'+'다음뉴스_20대이하남성_'+s_year+'-'+s+'.txt'
ff_30_w_name=f_dir+'다음뉴스_30대여성_'+s_year+'-'+s+'\\'+'다음뉴스_30대여성_'+s_year+'-'+s+'.txt'
ff_30_m_name=f_dir+'다음뉴스_30대남성_'+s_year+'-'+s+'\\'+'다음뉴스_30대남성_'+s_year+'-'+s+'.txt'
ff_40_w_name=f_dir+'다음뉴스_40대여성_'+s_year+'-'+s+'\\'+'다음뉴스_40대여성_'+s_year+'-'+s+'.txt'
ff_40_m_name=f_dir+'다음뉴스_40대남성_'+s_year+'-'+s+'\\'+'다음뉴스_40대남성_'+s_year+'-'+s+'.txt'
ff_50_w_name=f_dir+'다음뉴스_50대이상여성_'+s_year+'-'+s+'\\'+'다음뉴스_50대이상여성_'+s_year+'-'+s+'.txt'
ff_50_m_name=f_dir+'다음뉴스_50대이상남성_'+s_year+'-'+s+'\\'+'다음뉴스_50대이상남성_'+s_year+'-'+s+'.txt'
fc_name=f_dir+'다음뉴스_전체_'+s_year+'-'+s+'\\'+'다음뉴스_전체_'+s_year+'-'+s+'.csv'
fx_name=f_dir+'다음뉴스_전체_'+s_year+'-'+s+'\\'+'다음뉴스_전체_'+s_year+'-'+s+'.xls'

#크롬 드라이버를 사용해서 웹 페이지에 접속합니다.
chrome_path = "c:/temp/chromedriver_85/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)
        
#Step 4. 날짜 관련 작업하기
mon=["01","02","03","04","05","06","07","08","09","10","11","12"]

start_date2=[]
end_date2=[]

for s in range(int(s_year),int(e_year)+1):
    for i in range(0,len(mon)) :
        if mon[i] =="02" :
            sdate=str(s)+mon[i]+'01'
            start_date2.append(sdate)

            edate=str(s)+mon[i]+'28'
            end_date=edate
            end_date2.append(end_date)

        elif mon[i] == "04" or mon[i]=="06" or mon[i]=="09" or mon[i]=="11" :
            sdate=str(s)+mon[i]+'01'
            start_date2.append(sdate)

            edate=str(s)+mon[i]+'30'
            end_date=edate
            end_date2.append(end_date)
        else :
            sdate=str(s)+mon[i]+'01'
            start_date2.append(sdate)

            edate=str(s)+mon[i]+'31'
            end_date=edate
            end_date2.append(end_date)

#Step 5. 각 연령대별 뉴스 자료 수집하기
count_20_w = 0
count_20_m = 0
count_30_w = 0
count_30_m = 0
count_40_w = 0
count_40_m = 0
count_50_w = 0
count_50_m = 0

count2=[]    # 전체 게시물 번호 저장
age2=[]      # 연령 저장
gender2=[]   # 성별 저장
date2=[]     # 날짜 저장
content2=[]  # 뉴스 제목 저장

count = 1    # 전체 수집 건수를 카운트하는 변수

for x in range(0,len(end_date2)) :
    for i in range(int(start_date2[x]),int(end_date2[x])) :
        url = 'https://media.daum.net/ranking/age/?regDate=' 
        full_url = url+str(i)

        c_year=str(i)[0:4]
        c_mon=str(i)[4:6]
        c_day=str(i)[6:]
        
        driver.get(full_url)
        time.sleep(5)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
    
        article_result = soup.select('div.rank_news')
        print("%s년 %s월 %s일의 뉴스 수집 중" %(c_year,c_mon,c_day),"=" *49)

        for li in article_result:

            #20대 이하 여성 뉴스 모으기
            f1 = open(ff_20_w_name, 'a',encoding='UTF-8')
                
            w20_news = li.find('div',class_='item_age item_20s').find('div','rank_female').find('ol')
            print("*%s년 %s월 %s일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 =====" %(c_year,c_mon,c_day))
            no = 1
            for a in range(0,len(w20_news)-1) :
                              
                txt1=w20_news.find_all('a')
                try :
                    title1 = txt1[a].get_text()                   
                except :
                    continue
                else :
                    print(no,':',title1)           
                    count_20_w += 1 
                    f1.write(str(no)+':'+title1 + "\n") 
                    
                    count2.append(count)     # 전체 게시물 건수 
                    count += 1
                    
                    date2.append(i)          # 뉴스 날짜
                    age2.append('20대이하')  # 연령 
                    gender2.append('여성')   # 성별
                    content2.append(title1)    # 뉴스제목
                                       
                    no += 1
            f1.write("\n")    
            f1.close( )
        
            print("=" *80)
            print("\n")

            #20대 이하 남성 뉴스 모으기
            f2 = open(ff_20_m_name, 'a',encoding='UTF-8')
        
            m20_news = li.find('div',class_='item_age item_20s').find('div','rank_male').find('ol')

            print("*%s년 %s월 %s일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 =====" %(c_year,c_mon,c_day))
            no = 1
            for b in range(0,len(m20_news)-1) :
                
                txt2 = m20_news.find_all('a')
                try :
                    title2 = txt2[b].get_text() 
                except IndexError :
                    continue
                else :
                    print(no,':',title2)
                    count_20_m += 1
                    f2.write( str(no)+':'+title2 + "\n") 
                    
                    count2.append(count)
                    count += 1
                    
                    date2.append(i)          # 뉴스 날짜
                    age2.append('20대이하')  # 연령 
                    gender2.append('남성')   # 성별
                    content2.append(title2)    # 뉴스제목
                    
                    no += 1
            f2.write("\n")    
            f2.close( )
            print("=" *80)
            print("\n")

            #30대 여성 뉴스 모으기
            f3 = open(ff_30_w_name, 'a',encoding='UTF-8')
        
            w30_news = li.find('div',class_='item_age item_30s').find('div','rank_female').find('ol')

            print("*%s년 %s월 %s일의 뉴스 중 30대 여성이 많이 본 기사입니다 =====" %(c_year,c_mon,c_day))
            no = 1
            for c in range(0,len(w30_news)-1) :
                
                txt3 = w30_news.find_all('a')
                try :
                    title3 = txt3[c].get_text()                     
                except :
                    continue
                else :
                    print(no,':',title3)
                    count_30_w += 1
                    f3.write( str(no)+':'+title3 + "\n")
                    
                    count2.append(count)
                    count += 1
                    
                    date2.append(i)          # 뉴스 날짜
                    age2.append('30대')  # 연령 
                    gender2.append('여성')   # 성별
                    content2.append(title3)    # 뉴스제목

                    no += 1
            f3.write("\n")    
            f3.close( )
                               
            print("=" *80)
            print("\n")

            #30대 남성 뉴스 모으기
            f4 = open(ff_30_m_name, 'a',encoding='UTF-8')
        
            m30_news = li.find('div',class_='item_age item_30s').find('div','rank_male').find('ol')
    
            print("*%s년 %s월 %s일의 뉴스 중 30대 남성이 많이 본 기사입니다 ====" %(c_year,c_mon,c_day))
            no = 1
            for d in range(0,len(m30_news)-1) :
                
                txt4 = m30_news.find_all('a')
                try :
                    title4 = txt4[d].get_text() 
                except :
                    continue
                else :
                    print(no,':',title4)
                    count_30_m += 1
                    f4.write(str(no)+':'+ title4 + "\n")
                    
                    count2.append(count)
                    count += 1
                    
                    date2.append(i)          # 뉴스 날짜
                    age2.append('30대')  # 연령 
                    gender2.append('남성')   # 성별
                    content2.append(title4)    # 뉴스제목

                    no += 1
            f4.write("\n")
            f4.close( )
        
            print("=" *80)
            print("\n")

            #40대 여성 뉴스 모으기
            f5 = open(ff_40_w_name, 'a',encoding='UTF-8')
        
            w40_news = li.find('div',class_='item_age item_40s').find('div','rank_female').find('ol')

            print("*%s년 %s월 %s일의 뉴스 중 40대 여성이 많이 본 기사입니다 ====" %(c_year,c_mon,c_day))
            no =1
            for e in range(0,len(w40_news)-1) :
                
                txt5 = w40_news.find_all('a')
                try :
                    title5 = txt5[e].get_text()                  
                except :
                    continue
                else :
                    print(no,':',title5)
                    count_40_w += 1
                    f5.write( str(no)+':'+title5 + "\n")
                    
                    count2.append(count)
                    count += 1
                    
                    date2.append(i)          # 뉴스 날짜
                    age2.append('40대')  # 연령 
                    gender2.append('여성')   # 성별
                    content2.append(title5)    # 뉴스제목

                    no += 1
            f5.write("\n")
            f5.close( )
        
            print("=" *80)
            print("\n")

            #40대 남성 뉴스 모으기
            f6 = open(ff_40_m_name, 'a',encoding='UTF-8')
            m40_news = li.find('div',class_='item_age item_40s').find('div','rank_male').find('ol')

            print("*%s년 %s월 %s일의 뉴스 중 40대 남성이 많이 본 기사입니다 ====" %(c_year,c_mon,c_day))
            no =1
            for f in range(0,len(m40_news)-1) :
                
                txt6 = m40_news.find_all('a')
                try :
                    title6 = txt6[f].get_text()                    
                except :
                    continue
                else :
                    print(no,':',title6)
                    count_40_m += 1
                    f6.write( str(no)+':'+title6 + "\n")  
                    
                    count2.append(count)
                    count += 1
                    
                    date2.append(i)          # 뉴스 날짜
                    age2.append('40대')      # 연령 
                    gender2.append('남성')   # 성별
                    content2.append(title6)  # 뉴스제목

                    no += 1
            f6.write("\n")
            f6.close( )
        
            print("=" *80)
            print("\n")

            #50대 이상 여성 뉴스 모으기
            f7 = open(ff_50_w_name, 'a',encoding='UTF-8')
        
            w50_news = li.find('div',class_='item_age item_50s').find('div','rank_female').find('ol')

            print("*%s년 %s월 %s일의 뉴스 중 50대 여성이 많이 본 기사입니다 ====" %(c_year,c_mon,c_day))
            no =1
            for g in range(0,len(w50_news)-1) :
             
                txt7 = w50_news.find_all('a')
                try :
                    title7 =  txt7[g].get_text()                    
                except :
                    continue
                else :
                    print(no,':',title7)
                    count_50_w += 1
                    f7.write(str(no)+':'+ title7 + "\n")
                    
                    count2.append(count)
                    count += 1
                    
                    date2.append(i)          # 뉴스 날짜
                    age2.append('50대이상')  # 연령 
                    gender2.append('여성')   # 성별
                    content2.append(title7)  # 뉴스제목

                    no += 1
            f7.write("\n")
                
            f7.close( )
            print("=" *80)
            print("\n")
        
            #50대 이상 남성 뉴스 모으기
            f8 = open(ff_50_m_name, 'a',encoding='UTF-8')
        
            m50_news = li.find('div',class_='item_age item_50s').find('div','rank_male').find('ol')

            print("*%s년 %s월 %s일의 뉴스 중 50대 남성이 많이 본 기사입니다 ====" %(c_year,c_mon,c_day))
            no =1
            for h in range(0,len(m50_news)-1) :
            
                txt8 = m50_news.find_all('a')
                try :
                    title8 =  txt8[h].get_text()                    
                except :
                    continue
                else :
                    print(no,':',title8)
                    count_50_m += 1
                    f8.write(str(no)+':'+ title8 + "\n")
                    
                    count2.append(count)
                    count += 1
                    
                    date2.append(i)          # 뉴스 날짜
                    age2.append('50대이상')  # 연령 
                    gender2.append('남성')   # 성별
                    content2.append(title8)  # 뉴스제목

                    no += 1
            f8.write("\n")
            f8.close( )
        
            print("=" *80)
            print("\n")
        
            time.sleep(0.4)
                               
        if str(i) == en_date :
            break
            
    if str(i) == en_date :
            break

#step 6. csv , xls 형태로 저장하기
import pandas as pd

daum_news = pd.DataFrame()
daum_news['번호']=count2
daum_news['날짜']=pd.Series(date2)
daum_news['연령']=pd.Series(age2)
daum_news['성별']=pd.Series(gender2)
daum_news['뉴스기사']=pd.Series(content2)

# csv 형태로 저장하기
daum_news.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
daum_news.to_excel(fx_name ,index=False)

# Step 7.작업결과 요약 정보 보여주기
e_time = time.time( )
t_time = e_time - s_time            
print("=" *150)            
print("1.20대 이하 여성 : 총 %s 건  완료" %count_20_w)
print("2.20대 이하 남성 : 총 %s 건  완료" %count_20_m)
print("3.30대 여성      : 총 %s 건  완료" %count_30_w)
print("4.30대 남성      : 총 %s 건  완료" %count_30_m)
print("5.40대 여성      : 총 %s 건  완료" %count_40_w)
print("6.40대 남성      : 총 %s 건  완료" %count_40_m)
print("7.50대 이상 여성 : 총 %s 건  완료" %count_50_w)
print("8.50대 이상 남성 : 총 %s 건  완료" %count_50_m)
print("=" *150 )                                            
print("\n")
print("파일이 저장된 경로는 아래와 같습니다")
print("=" *150)
201print("2.20대 이하 남성: %s"  %ff_20_m_name)
print("3.30대      여성: %s"  %ff_30_w_name)
print("4.30대      남성: %s"  %ff_30_m_name)
print("5.40대      여성: %s"  %ff_40_w_name)
print("6.40대      남성: %s"  %ff_40_m_name)
print("7.50대 이상 여성: %s"  %ff_50_w_name)
print("8.50대 이상 남성: %s"  %ff_50_m_name)
print("9.csv 파일: %s"  %fc_name)
print("10.xls 파일: %s"  %fx_name)
print("=" *150)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))

driver.close( )

이 프로그램은 교육용 목적으로 서진수가 개발했으며 절대 상업용으로 사용해선 안됩니다
또한 이 프로그램으로 특정 회사에 피해를 줄 경우
모든 책임은 프로그램 사용자에게 있음을 미리 공지합니다
이 프로그램과 관련한 문의는 seojinsu@gmail.com 으로 보내주세요^^
다음 사이트 랭킹 뉴스중 연령별 랭킹 기사 정보 수집용 웹크롤러 by 서진수 


1.조회 시작일을 입력하세요(예:20160101) :  20190101
2.조회 종료일을 입력하세요(예:20161231) :  20190105
3.파일이 저장될 경로만 쓰세요;기본값:c:\temp\): 


요청하신 정보를 수집중이니 잠시만 기다려 주세요~~^^


2019년 01월 01일의 뉴스 수집 중 =================================================
*2019년 01월 01일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 =====
1 : [와우! 과학] 철통 보안이라던 '정맥 인증', '밀랍 손'이 뚫었다
2 : 하얗게 된 밤섬, 눈 아니었다..'가마우지' 배설물이었다
3 : 이영애·고현정도 줄 섰었는데..55년 만에 진료 중단, 왜?
4 : '세련된' 김정은, 남색 양복 입고 집무실 소파서 신년사 발표(종합)
5 : 0시에 새해 첫아기 힘찬 울음소리..자연분만 2.93kg 건강한 딸


*2019년 01월 01일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 =====
1 : '세련된' 김정은, 남색 양복 입고 집무실 소파서 신년사 발표(종합)
2 : 배우 이영애, 폐업위기 처한 제일병원 인수 나선다
3 : 하얗게 된 밤섬, 눈 아니었다..'가마우지' 배설물이었다
4 : 이영애·고현정도 줄 섰었는데..55년 만에 진료 중단, 왜?
5 : 커트가 3만2000원?.."직원들 다 내보낼 판" 최저임금 인상에 미용실 원장 한숨


*2019년 01월 01일의 뉴스 중 30대 여성이 많이 본 기사입니다 =====
1 : 서울 대형병원서 정신과 진료받던 환자가 의사 살

2019년 01월 04일의 뉴스 수집 중 =================================================
*2019년 01월 04일의 뉴스 중 20대 이하 여성이 많이 본 기사입니다 =====
1 : 전기차 충전하려다 살인미수까지..제주서 '충전 갈등' 심각
2 : 문 닫힌 편의점을 찾는 유일한 손님
3 : 英육군 "밤샘 게이머·셀카 중독자 구함"..Z세대 대상
4 : "내 아이에 '독성괴물' 사줬나".. 학부모들 패닉
5 : 임세원 교수 눈물 속 발인..병원 돌아보고 마지막 길


*2019년 01월 04일의 뉴스 중 20대 이하 남성이 많이 본 기사입니다 =====
1 : 중국, 핵폭탄 다음으로 강력한 초대형 폭탄 공개
2 : "내 아이에 '독성괴물' 사줬나".. 학부모들 패닉
3 : 英육군 "밤샘 게이머·셀카 중독자 구함"..Z세대 대상
4 : "평생 공짜식사라더니"..美남성, 버거킹 상대 소송
5 : 신재민 전날 3만자 분량 靑청원.."잘못을 저질렀다. 부끄러웠다"


*2019년 01월 04일의 뉴스 중 30대 여성이 많이 본 기사입니다 =====
1 : "기내 꼴불견 여전"..SNS에 진상 승객 공유하는 전직 승무원의 사연
2 : 전기차 충전하려다 살인미수까지..제주서 '충전 갈등' 심각
3 : 버스 개조한 캠핑카서 잠자던 일가족 5명 가스 중독..1명 중태(종합)
4 : 왕진진, 유흥업소 서비스 요구 거절되자 욕설
5 : "평생 공짜식사라더니"..美남성, 버거킹 상대 소송


*2019년 01월 04일의 뉴스 중 30대 남성이 많이 본 기사입니다 ====
1 : "평생 공짜식사라더니"..美남성, 버거킹 상대 소송
2 : 전기차 충전하려다 살인미수까지..제주서 '충전 갈등' 심각
3 : "혼선 유발 '한국식 나이' 대신 '만 나이'로 통일" 법안 발의
4 : 버스 개조한 캠핑카서 잠자던 일가족 5명 가스 중독..1명 중태(종합)
5 : 신재민 전날 3만자 분량 靑청원.."잘못을 저질렀다. 부끄러웠다"


*2019년 01월 